In [2]:
import requests
import pandas as pd
import time
from datetime import datetime
import numpy as np
import os
from surprise import SVD, accuracy # SVD model, 평가
from surprise import Reader, Dataset # SVD model의 dataset
import json
##apikey 매일 갱신 필요
api_key = 'RGAPI-d0afa7fa-f911-4241-abfc-f3264d981a65'#챔피언 리스트 가져오기
namelist = []
champlist='http://ddragon.leagueoflegends.com/cdn/12.1.1/data/en_US/champion.json'
json=requests.get(champlist)
champ= pd.DataFrame(json.json())
for name in champ.index:
    namelist.append(name)
supoter = ['Alistar', 'Amumu',  'Bard', 'Blitzcrank', 'Braum','Galio','Janna','Karma','Leona', 'Lulu','Maokai', 'Morgana', 'Nami', 'Nautilus', 'Rakan','Xerath', 'Rell', 'Senna', 'Seraphine', 'Sona', 'Soraka', 'Taric','Thresh', 'Yuumi', 'Zilean','Zyra','Shen']
jungle = ['Ekko', 'Elise', 'Evelynn','Fiddlesticks', 'Ivern','Gragas','Hecarim','JarvanIV','Karthus','Khazix', 'Kindred','LeeSin', 'Lillia','MasterYi','Nidalee', 'Nocturne', 'Nunu', 'Olaf','Poppy', 'Qiyana', 'Rammus', 'RekSai', 'Sejuani','Diana', 'Shaco', 'Shyvana', 'Skarner', 'Taliyah', 'Talon','Trundle', 'Udyr','Vi', 'Viego', 'Warwick','XinZhao', 'Zac']
top = ['Aatrox','Akali', 'Akshan', 'Cassiopeia', 'Chogath', 'Darius', 'DrMundo', 'Fiora', 'Gangplank', 'Garen', 'Gragas', 'Graves', 'Gwen', 'Heimerdinger', 'Illaoi', 'Irelia', 'Jax', 'Jayce', 'Kayle','Kennen','Kled','Lucian', 'Malphite', 'Maokai','MonkeyKing','Mordekaiser', 'Nasus','Olaf', 'Ornn', 'Poppy', 'Quinn', 'Renekton', 'Rengar', 'Riven', 'Rumble', 'Ryze', 'Sett', 'Shen', 'Singed', 'Sion', 'Sylas', 'TahmKench', 'Teemo', 'Trundle', 'Tryndamere', 'Urgot', 'Vayne', 'Viktor', 'Vladimir', 'Volibear', 'Yasuo', 'Warwick', 'Yorick', 'Zac']
mid = ['Ahri', 'Akali', 'Akshan', 'Anivia', 'Annie', 'AurelionSol', 'Azir', 'Cassiopeia', 'Corki', 'Ekko', 'Fizz', 'Galio', 'Heimerdinger', 'Irelia',  'Jayce','Kassadin', 'Katarina', 'Leblanc', 'Lissandra','Lucian', 'Lux',  'Malphite','Malzahar', 'Neeko','Orianna', 'Pantheon', 'Qiyana', 'Rumble', 'Ryze', 'Swain', 'Sylas', 'Syndra', 'Talon', 'TwistedFate', 'Veigar', 'Vex', 'Viktor', 'Vladimir', 'Yasuo', 'Yone', 'Zed', 'Zoe']
bottom = ['Aphelios', 'Ashe', 'Braum', 'Caitlyn', 'Corki', 'Draven', 'Ezreal', 'Jhin', 'Jinx', 'Kaisa', 'Kalista','KogMaw','Varus','Vayne','Lucian','MissFortune','Samira','Sivir', 'Xayah', 'Yasuo', 'Ziggs']
#프로 닉네임에서 ppuid가저오기



def get_puuid(user_name):
    #api url
    sohwan = "https://kr.api.riotgames.com/lol/summoner/v4/summoners/by-name/"+user_name+ "?api_key=" +  api_key
    r = requests.get(sohwan)
        #해당하는 puuid
    try:
        puuids = r.json()['puuid']
    except:
        pass
    return puuids

#리스트에 ppuid추가
def list_append_ppuids(names):
    puuids = []
    for name in names[0:4]:
        puuids.append(get_puuid(name))
    return puuids

#ppuid를 기반으로 게임 코드를 가져오고,중복을 제거함.
def Deduplication_game_list(puuids):
    game_lists = []
    game_list_ded = []#Deduplication의 약자 랍니다.
    for puuid in puuids:
        game_lists.append(get_game_list(puuid).json())
    for game_list in game_lists:
        for game_id in game_list:
            if game_id not in game_list_ded:
                game_list_ded.append(game_id)
    return game_list_ded

#게임리스트 가저오기
def get_game_list(puuid):
    try:
        count='50' #불러올 게임 수 최대 100
        game_list_url = 'https://asia.api.riotgames.com/lol/match/v5/matches/by-puuid/'+puuid+'/ids?start=0&count='+count+'&api_key='+api_key
        game_list =requests.get(game_list_url)
    except:
        pass
    return game_list

#게임 리스트에서 내용 가져오기
def get_game_info(game_list):
    i = 0
    game_info = []
    try:
        for ingame in game_list:
            game_info_url = 'https://asia.api.riotgames.com/lol/match/v5/matches/'+ingame+'?api_key='+api_key
            game_info_json=requests.get(game_info_url)
            game_info.append(game_info_json.json())
            i += 1
            time.sleep(0.2)
    except:
            pass
    return game_info


cnt = 1
def get_summnor_list(summoner_df):
    summnor_list = []
    for v in summoner_df.values:
        if v not in summnor_list:
            summnor_list.append(v)
    return summnor_list


def get_champ_list(champion_df):
    champion_list =[]
    for v in champion_df.values:
        if v not in champion_list:
            champion_list.append(v)
    return champion_list

def data_trance():
    #가공할 CSV를 폴더에서 찾아서 하나의 데이터프레임으로 가공
    path = 'target/' 
    path2 = 'target/clear/'
    file_list = os.listdir(path)
    file_list_csv = [file for file in file_list if  file.endswith('.csv')]
    print(file_list_csv)
    file_name = file_list_csv
    test_df = pd.DataFrame()
    for i in file_list_csv:
        data = pd.read_csv(path + i)
        try:
            os.rename(path + i ,path2 + i)
        except:
            os.remove(path2 + i)
            os.rename(path + i ,path2 + i)
        test_df = pd.concat([test_df,data])
    
    #na값과,중복제거
    test_df.dropna(axis=0)
    test_df.drop_duplicates(inplace=True)

    #외부의 방법으로 데이터 병합시 에러가 났을때 제거
    try:
        for index in test_df[test_df['assists']=='assists'].index:
            test_df.drop(test_df.index[index],inplace=True)
    except:
        pass
    #오브젝트를 숫자형으로 변경
    test_df['assists']=test_df['assists'].astype('str').astype('int64')
    test_df['deaths']=test_df['deaths'].astype('str').astype('int64')
    test_df['kills']=test_df['kills'].astype('str').astype('int64')

    #승리시 점수에 Ture False를 1과 0으로 수정
    test_df.win =  test_df.win.apply(lambda x : 1 if x =='True' else 0)
    #0데스의 경우 20퍼센트의 가산점 부여
    test_df.deaths =  test_df.deaths.apply(lambda x : 0.8 if x ==0 else x)
    # 인덱스 새로 생성
    test_df.reset_index(drop=True,inplace=True)
    #만약 서포터의 경우 어시스트가 더 높기때문에 어시스트의 10퍼센트 패널티 부여
    test_df['Rating']=test_df["Rating"] = np.where((test_df["championName"].isin(supoter)), ((test_df['kills']+(test_df['assists']*0.9)/test_df['deaths'])+test_df['win']) ,
                                                           ((test_df['kills']+(test_df['assists'])/test_df['deaths'])+test_df['win'])  )
    #챔피언별,소환수별 분리
    summoner_df = test_df['summonerName']
    champion_df= test_df['championName']
    
    cnt = 0
    point = 0
    chams_point = []
    chams = []
    id = []
    play_count = []
    out_put = pd.DataFrame()
    #게임마다 중복 플레이어가 있을수도 있고 없을 수도 있고,똑같은 캐릭터 플레이가 있을수도 있고 없을 수도있어서 모든데이터를 재정렬한다.
    for names in get_summnor_list(summoner_df):
        championNames = test_df[test_df['summonerName']==names]['championName']
        for chaps in get_champ_list(championNames):
            mi = test_df[(test_df['summonerName']==names)&(test_df['championName']==chaps)]
            rating = 0
            cnt = 0
            for ratings in mi.Rating:
                rating += ratings
                cnt += 1
            chams_point.append(rating/cnt)
            id.append(names)
            chams.append(chaps)
            play_count.append(cnt)
    out_put['id'] = id
    out_put['chams'] = chams
    out_put['chams_point'] = chams_point
    out_put['play_count'] = play_count
    #트랜스 폴더에 저장한다.
    out_put.to_csv('surprise/surprise_target/{0}'.format(file_list_csv[0]),encoding='utf-8-sig')

def get_user_data(nick_name):
    now = datetime.now()
    time.sleep(1.5)
    ppuid=get_puuid(nick_name)
    time.sleep(1.5)
    game_list = get_game_list(ppuid).json()
    time.sleep(1.5)
    game_infos = get_game_info(game_list)

    matchId =[]
    summonerName =[]
    championName = []
    teamId = []
    timePlayed = []
    individualPosition = []
    win = []
    visionScore = []
    kills = []
    assists = []
    deaths = []
    player_by_champ_list=pd.DataFrame()
    for game_info in game_infos:
        try:
            if (game_info['info']['queueId']==420): #솔랭만 저장
                   for player_info in game_info['info']['participants']:
                        if(player_info['summonerName']==nick_name):
                            matchId.append(game_info['metadata']['matchId'])
                            summonerName.append(player_info['summonerName'])
                            championName.append(player_info['championName'])
                            individualPosition.append(player_info['individualPosition'])
                            win.append(player_info['win'])
                            assists.append(player_info['assists'])
                            visionScore.append(player_info['visionScore'])
                            deaths.append(player_info['deaths'])
                            kills.append(player_info['kills'])
                            if player_info['teamId']==100:
                                teamId.append('blue')
                            else:
                                teamId.append('red')
                            timePlayed.append(player_info['timePlayed'])
                        else:
                            pass
            else:
                 pass
        except:
            pass
    player_by_champ_list['summonerName']= summonerName
    player_by_champ_list['championName']= championName
    player_by_champ_list['matchId'] = matchId
    player_by_champ_list['teamId'] = teamId
    player_by_champ_list['timePlayed'] =timePlayed
    player_by_champ_list['individualPosition'] = individualPosition
    player_by_champ_list['win']= win
    player_by_champ_list['visionScore'] = visionScore
    player_by_champ_list['assists'] = assists
    player_by_champ_list['deaths'] = deaths
    player_by_champ_list['kills'] = kills

    player_by_champ_list.to_csv('target/{0}.csv'.format(nick_name),encoding='utf-8-sig')

def surprise(nickname,position):
    #데이터 추천을 위한 프리셋 불러오기
    rating =  pd.read_csv('surprise/surprise_pitdata/pitdata.csv',encoding='utf-8-sig')
    #추천받을 유저의 데이터 불러오기
    target = pd.read_csv('surprise/surprise_target/{0}.csv'.format(nickname),encoding='utf-8-sig')
    #두개를 하나의 데이터프레임으로 만들기
    rating=pd.concat([rating,target])
    #불필요 컬럼 삭제
    rating.drop('Unnamed: 0',axis=1,inplace=True)
    #중복 데이터 제거
    rating.drop_duplicates(inplace=True)
    #불필요 데이터 제거
    rating.drop('play_count',axis=1,inplace=True)
    #유저의 정보를 추가하여 새로운 프리셋으로 저장
    rating.to_csv('surprise/surprise_pitdata/pitdata.csv')

    #정규화
    df=rating.chams_point
    normalization_df = (df - df.mean())/df.std()
    rating.chams_point = normalization_df


    #분석을 위한 데이터 가공
    tab = pd.crosstab(rating['id'], rating['chams'])
    # 행렬구조로 변환
    rating_g = rating.groupby(['id', 'chams'])
    rating_g.sum()
    tab = rating_g.sum().unstack()
    #분석때 필요한 점수 설정
    reader = Reader(rating_scale= (-2, 2)) # 평점 범위
    data = Dataset.load_from_df(df=rating, reader=reader)
    #데이터셋
    train = data.build_full_trainset() # 훈련셋
    test = train.build_testset() # 검정셋
    #추천 대상자
    user_id = nickname # 추천대상자
    item_ids = namelist # 추천 챔피언
    actual_rating = 0 # 실제 평점
    #학습 시작
    model = SVD(n_factors=100, n_epochs=20, random_state=123)
    model.fit(train) # model 생성
    predict_list = []
    for item_id in item_ids :
        predict_list.append(model.predict(user_id, item_id, actual_rating))
        cu_df = pd.DataFrame(predict_list).sort_values(by='est',ascending=False)

    #포지션별 가산점 추가
    if (position=='jungle'):
        cu_df.loc[cu_df['iid'].isin(jungle),'est'] = (cu_df[cu_df['iid'].isin(jungle)].est+10)
    elif (position=='supoter'):
        cu_df.loc[cu_df['iid'].isin(supoter),'est'] = (cu_df[cu_df['iid'].isin(supoter)].est+10)
    elif (position=='top'):
        cu_df.loc[cu_df['iid'].isin(top),'est'] = (cu_df[cu_df['iid'].isin(top)].est+10)
    elif (position=='mid'):
        cu_df.loc[cu_df['iid'].isin(mid),'est'] = (cu_df[cu_df['iid'].isin(mid)].est+10)
    elif (position=='bottom'):
        cu_df.loc[cu_df['iid'].isin(bottom),'est'] = (cu_df[cu_df['iid'].isin(bottom)].est+10)
    cu_df= cu_df.sort_values(by='est',ascending=False)
    q1= rating[rating.id==nickname].chams.values# 플레이한 챔피언 목록
    q2 = cu_df.iid.isin(q1)==False # 플레이한 쳄피언을 제외
    most = target.sort_values('play_count',ascending=False).head(3).chams.reset_index(drop=True) #추천 챔피언 3개
    suggestion =cu_df[q2].iid.head(3).reset_index(drop=True) #그중에서 성능이 좋은것 3개
    final=pd.concat([suggestion,most],ignore_index=True)
    return final.to_json()

In [ ]:
from flask import Flask, request, jsonify,redirect
import json
class MyFlask(Flask):
    def make_response(self, rv):
        if not rv:
            return super().make_response("Any String")
        return super().make_response(rv) 
app = MyFlask(__name__)
app.config['JSON_AS_ASCII'] = False
@app.route('/')
def main():
    return 'hello'

@app.route('/param')
def hello():
    po = request.args.get('po')
    a = request.args.get('name')
    new_s = a.replace('+',' ')
    get_user_data(new_s)
    data_trance()
    js = surprise(new_s,po)
    print(js)
    json_get =  json.loads(js)
    print(json_get)
    url = 'http://localhost:8080/fpj/champion.do?s1={0}&s2={1}&s3={2}&m1={3}&m2={4}&m3={5}'.format(json_get['0'],json_get['1'],json_get['2'],json_get['3'],json_get['4'],json_get['5'])
    print(url)
    return redirect(url)

if __name__== "__main__":
    app.run()

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


['넴모넴모뭐믹.csv']


127.0.0.1 - - [26/Jan/2022 17:03:13] "GET /param?po=jungle&name=넴모넴모뭐믹 HTTP/1.1" 302 -


{"0":"Karthus","1":"Evelynn","2":"Qiyana","3":"Sona","4":"Talon","5":"Nocturne"}
{'0': 'Karthus', '1': 'Evelynn', '2': 'Qiyana', '3': 'Sona', '4': 'Talon', '5': 'Nocturne'}
http://localhost:8080/fpj/champion.do?s1=Karthus&s2=Evelynn&s3=Qiyana&m1=Sona&m2=Talon&m3=Nocturne
